In [1]:
import tensorflow as tf

import numpy as np
from numpy import array
from pickle import dump
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from pickle import load
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
data = open('shakespear.txt', 'r').read()
data_out = data[1:]+data[0]
chars = sorted(list(set(data)))
char2num = {ch:i for i,ch in enumerate(chars)}
num2char = {i:ch for i,ch in enumerate(chars)}
vocab_size = len(chars)
input_one_hot = np.zeros([len(data), len(chars)])
for i in range(len(data)):
    input_one_hot[i,char2num[data[i]]] =1



seq_length = 10
batch_size = len(data)
hidden_size = 20
TRAINING_ITERS = 1
epoch = 1
TRAINING_ITERS = 1
def generate_data():
    pos = np.random.choice(range(seq_length))
    batch_x = []
    batch_y = []
    for _ in range(batch_size):
        if pos+seq_length+1 >= len(input_one_hot):
            pos = np.random.choice(range(seq_length)) # go back to random starting point
        seq_x = input_one_hot[pos:pos+seq_length]
        seq_y = input_one_hot[pos+seq_length]
        batch_x.append(seq_x)
        batch_y.append(seq_y)
        pos += 1
    batch_x = np.array(batch_x)
    batch_y = np.array(batch_y)
    return batch_x,batch_y
    #print(batch_x.shape)
X,Y = generate_data()
print(X.shape)



(99993, 10, 62)


In [3]:

# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
loss_fn = lambda y_true, y_pred: tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

print(model.summary())
# compile model
model.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])
# fit model



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 75)                41400     
_________________________________________________________________
dense (Dense)                (None, 62)                4712      
Total params: 46,112
Trainable params: 46,112
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
print(X.shape,Y.shape)

(99993, 10, 62) (99993, 62)


In [ ]:
model.fit(X, Y, epochs=50, verbose=2,initial_epoch=24)

#for i in range(TRAINING_ITERS):
#    model.partial_fit(X,Y,verbose=2)

# save the model to file
#model.save('model.h5')
# save the mapping
#dump(mapping, open('mapping.pkl', 'wb'))

Epoch 25/50
 - 35s - loss: 1.4227 - acc: 0.5668
Epoch 26/50
 - 33s - loss: 1.4129 - acc: 0.5687
Epoch 27/50
 - 33s - loss: 1.4032 - acc: 0.5725
Epoch 28/50
 - 34s - loss: 1.3959 - acc: 0.5744
Epoch 29/50
 - 33s - loss: 1.3877 - acc: 0.5760
Epoch 30/50
 - 34s - loss: 1.3792 - acc: 0.5791
Epoch 31/50
 - 34s - loss: 1.3714 - acc: 0.5806
Epoch 32/50
 - 32s - loss: 1.3641 - acc: 0.5834
Epoch 33/50
 - 34s - loss: 1.3584 - acc: 0.5837
Epoch 34/50
 - 34s - loss: 1.3517 - acc: 0.5863
Epoch 35/50
 - 33s - loss: 1.3448 - acc: 0.5871
Epoch 36/50
 - 33s - loss: 1.3381 - acc: 0.5895
Epoch 37/50
 - 33s - loss: 1.3314 - acc: 0.5907
Epoch 38/50
 - 34s - loss: 1.3261 - acc: 0.5923
Epoch 39/50
 - 33s - loss: 1.3202 - acc: 0.5943
Epoch 40/50
 - 34s - loss: 1.3142 - acc: 0.5964
Epoch 41/50
 - 33s - loss: 1.3098 - acc: 0.5973
Epoch 42/50
 - 32s - loss: 1.3036 - acc: 0.5994
Epoch 43/50
 - 33s - loss: 1.2980 - acc: 0.6013
Epoch 44/50
 - 32s - loss: 1.2938 - acc: 0.6008
Epoch 45/50


In [15]:
tensorboard --logdir=path/to/log-directory

SyntaxError: can't assign to operator (<ipython-input-15-1a919625dfe2>, line 1)

In [13]:
 
#generate a sequence of characters with a language model
def generate_seq(model, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [char2num[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        encoded = to_categorical(encoded, num_classes=vocab_size)

        
        # reverse map integer to character
        #out_char = ''
        if in_text[-1] == ' ' or in_text[-1] =='\n' or in_text[-1]==':':
        #if True:
            probs = model.predict_proba(encoded, verbose=0)
            yhat = np.random.choice(Y.shape[1],p=probs[0])
            #print(yhat)
            char = num2char[yhat]
            in_text += char
        else:
            yhat = model.predict_classes(encoded,verbose=0)
            #print(yhat)
            char = num2char[yhat[0]]
            in_text += char   
    return in_text


print(generate_seq(model, 10, 'T', 10000))

TOLPIO:
Narry, my!e
To make thy offenve and hunver'st dray I much live
O, that in down like a letst thou hast ither by as greature, ta bear,
He, I will seaw
We preet as life; and, and ream ublesing make oye
In my bear;
To epalors and he's a mistragagf lord, fixed;
And you muty wrate me hy dow
To.

CRINCLANL:
Deke but seen his same: the here, and doing and her faliat before
That I shall be bant:
And
they must;
Fear him full more takest that heard Sirnon that ever yet went is vilyare: shall nome the most are her:
Of knew poor dreven.

TROILUS:
A can you both
And heard fear-king with wixe and sid to Ruched his hin,
And I so; Ant Illake with thouns him the paberest subjectsoance, in then flower-bairs,
Indle see with thee, I must deer nider to beds inderun,
Who
OR I do them from the fallent!

ENY SIOLO:
I my, why not Tribut!
What it that thou not band given mefork to deer me of her begnest from the abjedit to thee, thrue yot wroty.

PISARUS:
I
beseevery therefile oid twalses,
And thou point

In [40]:
import numpy as np
in_text = 'hello worl'
encoded = [mapping[char] for char in in_text]
encoded = pad_sequences([encoded], maxlen=10, truncating='pre')
encoded = to_categorical(encoded, num_classes=len(mapping))
yhat = model.predict_proba(encoded, verbose=0)
print(yhat[0])
print(mapping.items())
np.random.choice(y.shape[1],p=yhat[0])

[9.55516655e-12 3.43545293e-03 4.37857561e-05 2.95649306e-03
 1.32304398e-04 1.32646455e-05 2.34582811e-04 5.97190319e-06
 1.77629539e-04 2.36619017e-05 3.37657667e-07 2.80047118e-07
 3.12416604e-09 2.98170448e-08 3.29765228e-08 7.28715421e-11
 3.31476201e-11 1.75440988e-07 7.68207045e-08 9.40990503e-07
 3.78405751e-09 6.86346224e-09 1.07870992e-08 1.10448298e-07
 5.05748261e-08 3.73938008e-07 3.63012009e-10 4.71070400e-08
 5.50160166e-06 4.13635632e-08 1.73824366e-09 5.66055064e-07
 1.11106101e-06 8.68983534e-07 1.72449120e-07 1.71315033e-08
 3.19471210e-03 3.94385324e-06 3.95673960e-05 7.38599777e-01
 3.22828069e-03 1.20052464e-04 2.15253181e-06 8.39210674e-03
 8.00172682e-04 4.14027738e-07 7.28923408e-03 9.18784663e-02
 3.03084757e-02 1.09101973e-04 2.78693711e-04 8.95698031e-05
 8.26176063e-07 2.70100918e-05 9.33347717e-02 1.23015570e-03
 6.71466161e-03 1.36334875e-05 1.36025716e-04 1.75431527e-08
 7.17429956e-03 5.03732542e-11]
dict_items([('\n', 0), (' ', 1), ('!', 2), ("'", 3), 

39

(9999, 62)


In [23]:
model = Sequential()
model.add(LSTM(75, input_shape=(batch_x.shape[1], batch_x.shape[2])))
model.add(Dense(vocab_size, input_shape = (50,20), activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 75)                41400     
_________________________________________________________________
dense_2 (Dense)              (None, 62)                4712      
Total params: 46,112
Trainable params: 46,112
Non-trainable params: 0
_________________________________________________________________
None


In [24]:

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(batch_x, batch_y, epochs=100, verbose=2)

ValueError: Error when checking target: expected dense_2 to have 2 dimensions, but got array with shape (2, 50, 62)